# AWS Marketplace Product Usage Demonstration - topicmodelling using bertopic Model Package

##topicmodelling using bertopic

This sample notebook demonstrates the usage of the topicmodelling using bertopic.
This solution is a superviser learning model to detect the Dominant topic and probabaility of that textual transcriptions

This sample notebook shows you how to deploy topicmodelling using bertopic using Amazon SageMaker.

**Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to DefenSight Botnet Detector. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](# Subscribe-to-the-model-package)
2. [Set up the environment](# Setup the environment)
3. [Create the session](# Create the session)
4. [Create Model](# Create model)
5. [Create an endpoint and perform real-time inference](#-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
6. [Perform batch inference](# Perform-batch-inference)
    1. [Configure the input S3 bucket folder](# A. -Configure the input S3 bucket folder)
    2. [Deploy the model](# B. -Deploy the model)
    3. [Download the file from output S3 bucket folder](# C. -Download the file from output S3 bucket folder)
    4. [Visualize data](# D. -Visualize data)
7. [Clean-up](# Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page DefenSight Botnet Detector. 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## 2. Set up the environment

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from sagemaker.serializers import CSVSerializer
import boto3
import json

role = get_execution_role()

# S3 prefixes
common_prefix = "<s3 bucket name>"
batch_inference_input_prefix = "topic/input"
batch_inference_output_prefix = "topic/output"
from sagemaker.predictor import Predictor

## 3. Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our Amazon SageMaker operations.

In [3]:
sagemaker_session = sage.Session()

## 4. Create Model

Now we use the above Model Package to create a model

In [4]:
model_package_arn = '<provide your model-package arn>'

In [5]:
model_name = 'topic-model'

content_type = 'application/json'

real_time_inference_instance_type = 'ml.t2.medium'
batch_transform_inference_instance_type = 'ml.m5.large'

In [6]:
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session, content_type)

# Create a deployable model from the model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls = predict_wrapper)

## 5. Create-an-endpoint-and-perform-real-time-inference

#### A. Create an endpoint

In [7]:
#Deploy the model
predictor = model.deploy(1,real_time_inference_instance_type,endpoint_name = model_name)

-------------------------------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [8]:
file_name = 'input/test.json'

#### C. Perform real-time inference

In [25]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    output.txt

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [26]:
# import json
# with open("output.txt", 'r') as f:
#     output = json.load(f)
# print(json.dumps(output,indent=1))
    
import json  
  
with open('output.txt', 'r') as f:  
    content = f.read()  
    content = content.replace("'", '"')  
    output = json.loads(content)  
  
print(json.dumps(output, indent=1))  

    

{
 "topics": [
  "Topic 0: equation, flow, of, prove, mathbb, algebra, we, solution, group, be",
  "Topic 1: magnetic, phase, spin, quantum, state, temperature, transition, inside, of, field",
  "Topic 2: galaxy, star, mass, of, planet, at, with, stellar, and, be",
  "Topic 3: network, neural, deep, layer, weight, train, activation, architecture, training, accuracy",
  "Topic 4: convergence, stochastic, gradient, convex, optimization, algorithm, descent, method, nonconvex, rate",
  "Topic 5: camera, 3d, object, localization, slam, pose, scene, sensor, point, map",
  "Topic 6: channel, user, antenna, transmit, wireless, relay, communication, scheme, capacity, transmission",
  "Topic 7: causal, treatment, outcome, effect, trial, inference, study, method, clinical, test",
  "Topic 8: speech, speaker, recognition, music, audio, train, neural, acoustic, model, feature",
  "Topic 9: bandit, regret, arm, algorithm, online, bound, multiarmed, problem, reward, auction",
  "Topic 10: graph, vert

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [27]:
predictor.delete_endpoint(delete_endpoint_config = True)

## 6. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

### A. Configure the input S3 bucket folder

In [28]:
transform_input_folder = 'input'
transform_input = sagemaker_session.upload_data(transform_input_folder,common_prefix,batch_inference_input_prefix)

### B. Deploy the model

In [29]:
transformer = model.transformer(1,batch_transform_inference_instance_type,output_path="s3://"+common_prefix+"/"+batch_inference_output_prefix+"/")
transformer.transform(transform_input,content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating transform job with name: berttopic-2023-10-05-08-39-23-875


...................................................Starting the inference server with 2 workers.
[2023-10-05 08:47:54 +0000] [10] [INFO] Starting gunicorn 21.2.0
[2023-10-05 08:47:54 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2023-10-05 08:47:54 +0000] [10] [INFO] Using worker: sync
[2023-10-05 08:47:54 +0000] [13] [INFO] Booting worker with pid: 13
[2023-10-05 08:47:54 +0000] [14] [INFO] Booting worker with pid: 14
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.8/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument 

### C. Download the file from output S3 bucket folder

In [30]:
import os
s3_conn = boto3.client("s3")
with open('output2.txt','wb') as f:
    s3_conn.download_fileobj(common_prefix,batch_inference_output_prefix+'/'+'test.json.out',f)
    print("Output file loaded from bucket")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Output file loaded from bucket


### D. Visualize data

In [31]:
with open('output2.txt','r') as f:
    content = f.read()  
    content = content.replace("'", '"')  
    output = json.loads(content)  
  
print(json.dumps(output, indent=1)) 

{
 "topics": [
  "Topic 0: equation, flow, of, prove, mathbb, algebra, we, solution, group, be",
  "Topic 1: magnetic, phase, spin, quantum, state, temperature, transition, inside, of, field",
  "Topic 2: galaxy, star, mass, of, planet, at, with, stellar, and, be",
  "Topic 3: network, neural, deep, layer, weight, train, activation, architecture, training, accuracy",
  "Topic 4: convergence, stochastic, gradient, convex, optimization, algorithm, descent, method, nonconvex, rate",
  "Topic 5: camera, 3d, object, localization, slam, pose, scene, sensor, point, map",
  "Topic 6: channel, user, antenna, transmit, wireless, relay, communication, scheme, capacity, transmission",
  "Topic 7: causal, treatment, outcome, effect, trial, inference, study, method, clinical, test",
  "Topic 8: speech, speaker, recognition, music, audio, train, neural, acoustic, model, feature",
  "Topic 9: bandit, regret, arm, algorithm, online, bound, multiarmed, problem, reward, auction",
  "Topic 10: graph, vert

### 4. Clean-up

### A. Delete the model

In [32]:
model.delete_model()

INFO:sagemaker:Deleting model with name: berttopic-2023-10-05-08-39-21-096


#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.